In [ ]:
import re

with open('F.txt', 'r', encoding='utf-8') as f:
    content = f.read()

import json



token_pattern = re.compile(r'''
    (\\begin\{[^{}]*\}|\\end\{[^{}]*\}    
    | \\[a-zA-Z]+                         
    | \\\\                               
    | \\[{}#%&_$|]                        
    | \\[!,:;]                            
    | \'[\^]?                             
    | \^                                  
    | \_                                  
    | \n                                  
    | \s+                                 
    | [-+*=&/%<>!?.,;:'"()]               
    | [0-9]                               
    | \$\$                                
    | \S)                                 
''', re.VERBOSE)


token_to_id = {  
    "PAD": 0,  
    "BOS": 1,  
    "EOS": 2
}  
  
tokens = set(token_pattern.findall(content))
  
for idx, token in enumerate(sorted(tokens), start=3):  
    token_to_id[token] = idx  
  
id_to_token = {idx: token for token, idx in token_to_id.items()}  
  
with open('token_to_id.json', 'w', encoding='utf-8') as json_file:  
    json.dump(token_to_id, json_file, ensure_ascii=False, indent=4)

In [ ]:
import json

class Words:
    def __init__(self, token_to_id_path):
        with open(token_to_id_path, 'r', encoding='utf-8') as json_file:
            self.token_to_id = json.load(json_file)
        print(f"共有{len(self.token_to_id)}个token")

    def __len__(self):
        return len(self.token_to_id)

    def encode(self, tokens_list):
        encoded_inputs = {'input_ids': [], 'attention_mask': []}
        for tokens in tokens_list:
            number_list = []
            i = 0
            while i < len(tokens):
                match = None
                for j in range(len(tokens), i, -1):
                    substr = tokens[i:j]
                    if substr in self.token_to_id:
                        match = substr
                        break
                if match:
                    number_list.append(self.token_to_id[match])
                    i += len(match)
                else:
                    number_list.append(self.token_to_id.get(tokens[i], self.token_to_id.get("UNK", 3)))
                    i += 1


            encoded_inputs['input_ids'].append(number_list)
            encoded_inputs['attention_mask'].append([1] * len(number_list))
        
        return encoded_inputs